In [ ]:
import random
import time
import threading
import queue
import os


latest_temperatures = {}
temperature_averages = {}


temperature_queue = queue.Queue()


data_lock = threading.RLock()  
condition = threading.Condition()  

def simulate_sensor(sensor_id):
    while True:
        temp = random.randint(15, 40)
        
        with data_lock:
            latest_temperatures[sensor_id] = temp
        
        with condition:
            temperature_queue.put((sensor_id, temp))
            condition.notify() 
        time.sleep(1)

def process_temperatures():
    temp_records = {}

    while True:
        with condition:
            while temperature_queue.empty():
                condition.wait()  # Wait until new data arrives

            sensor_id, temp = temperature_queue.get()

        with data_lock:
            if sensor_id not in temp_records:
                temp_records[sensor_id] = []
            
            temp_records[sensor_id].append(temp)
            
            if len(temp_records[sensor_id]) > 10:
                temp_records[sensor_id].pop(0)

            avg_temp = sum(temp_records[sensor_id]) / len(temp_records[sensor_id])
            temperature_averages[sensor_id] = round(avg_temp, 2)

def initialize_display():
    os.system("cls" if os.name == "nt" else "clear")
    print("Current temperatures:")
    sensors = [f"Sensor {i}: --°C" for i in range(3)]
    print("Latest Temperatures: " + " ".join(sensors))
    
    for i in range(3):
        print(f"Sensor {i+1} Average: --°C")

def update_display():
    os.system("cls" if os.name == "nt" else "clear")
    print("Current temperatures:")
    
    with data_lock:
        sensor_data = [f"Sensor {i}: {latest_temperatures.get(f'Sensor-{i+1}', '--')}°C" for i in range(3)]
        print("Latest Temperatures: " + " ".join(sensor_data))

        for i in range(3):
            avg_temp = temperature_averages.get(f"Sensor-{i+1}", "--")
            print(f"Sensor {i+1} Average: {avg_temp}°C")


sensor_threads = []
num_sensors = 3

for i in range(num_sensors):
    sensor_id = f"Sensor-{i+1}"
    sensor_thread = threading.Thread(target=simulate_sensor, args=(sensor_id,), daemon=True)
    sensor_threads.append(sensor_thread)
    sensor_thread.start()

processing_thread = threading.Thread(target=process_temperatures, daemon=True)
processing_thread.start()

initialize_display()
while True:
    update_display()
    time.sleep(1)




Current temperatures:
Latest Temperatures: Sensor 0: --°C Sensor 1: --°C Sensor 2: --°C
Sensor 1 Average: --°C
Sensor 2 Average: --°C
Sensor 3 Average: --°C
Current temperatures:
Latest Temperatures: Sensor 0: 35°C Sensor 1: 39°C Sensor 2: 26°C
Sensor 1 Average: 35.0°C
Sensor 2 Average: 39.0°C
Sensor 3 Average: 26.0°C
Current temperatures:
Latest Temperatures: Sensor 0: 26°C Sensor 1: 23°C Sensor 2: 23°C
Sensor 1 Average: 30.5°C
Sensor 2 Average: 31.0°C
Sensor 3 Average: 24.5°C
Current temperatures:
Latest Temperatures: Sensor 0: 28°C Sensor 1: 26°C Sensor 2: 26°C
Sensor 1 Average: 29.67°C
Sensor 2 Average: 29.33°C
Sensor 3 Average: 25.0°C
Current temperatures:
Latest Temperatures: Sensor 0: 35°C Sensor 1: 23°C Sensor 2: 32°C
Sensor 1 Average: 31.0°C
Sensor 2 Average: 27.75°C
Sensor 3 Average: 26.75°C
